### 포켓몬 마스터가 되자

In [5]:
import requests  # HTTP 요청을 보내기 위한 requests 라이브러리 불러오기
import re
import os
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# 환경 변수 가져오기
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
ai_search_endpoint = os.getenv("AI_SEARCH_ENDPOINT")
ai_search_api_key = os.getenv("AI_SEARCH_API_KEY")
ai_search_index_name = os.getenv("AI_SEARCH_INDEX")
ai_search_semantic = os.getenv("AI_SEARCH_SEMANTIC")
system_content = os.getenv("SYSTEM_CONTENT")

def request_gpt(prompt):
    # 요청 헤더 설정
    headers = {
        "Content-Type": "application/json",  # 요청 본문을 JSON 형식으로 지정
        "api-key": api_key  # 인증을 위한 API 키 포함
    }

    # 요청 본문 (채팅 메시지 설정)
    body = {
        "messages": [
            {
                "role": "system",
                "content": system_content
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        "temperature": 0.7,  # 창의성 조절 (0: 보수적, 1: 창의적)
        "top_p": 0.9,  # 확률 분포 기반 단어 선택 (높을수록 다양한 단어 선택 가능)
        "max_tokens": 4096,
        "stop":None,  
        "stream":False,
        "data_sources": [
            {
            "type": "azure_search",
            "parameters": {
                "endpoint": ai_search_endpoint,
                "index_name": ai_search_index_name,
                "semantic_configuration": ai_search_semantic,
                "query_type": "semantic",
                "fields_mapping": {},
                "in_scope": True,
                "filter": None,
                "strictness": 2,
                "top_n_documents": 8,
                "authentication": {
                "type": "api_key",
                "key": ai_search_api_key
                },
                "key": ai_search_api_key,
            }
            }
        ],
    }


    # OpenAI API에 POST 요청을 보내고 응답을 받아옴
    response = requests.post(endpoint, headers=headers, json=body)
    if response.status_code == 200:
        response_json = response.json()  # 응답을 JSON 형식으로 변환

        # AI의 응답에서 채팅 메시지 추출
        message = response_json['choices'][0]['message']
        citation_list = message['context']['citations']

        # 부적절한 콘텐츠 관리 필터에 걸린 게 있는지 그 결과를 출력하는 것. 수업에서는 활용 안함.
        # content_filter = response_json['choices'][0]['content_filter_results']
        # 역할 (예: assistant)
        role = message['role']
        # 응답 메시지 (AI의 답변)
        content = message['content']
        content = re.sub(r'\[doc(\d+)\]', r'[참조 \1]', content)
        # 역할 출력 (예: assistant)
        # print(role)
        return content, citation_list
    else:
        return "", list()
    
# 사용자 입력 프롬프트
# content = request_gpt("안녕 네 이름 뭐야?")
content, citation_list = request_gpt("잠만보")

# Jupyter Notebook 환경에서 마크다운 형식으로 AI 응답 표시
from IPython.display import Markdown
display(Markdown(content))

for index, c in enumerate(citation_list, start=1):
    lines = c['content'].splitlines()
    print(f"Index : {index} {lines[-1]} {lines[-2]}번 {lines[0]} {lines[1]}")

🔥 친구, 잠만보(Snorlax)에 대해 궁금하구나! 그럼 내가 최고로 멋지게 알려줄게! 🔥  

### 잠만보의 정보:
- **타입**: 노말(Normal)
- **설명**: 잠만보는 항상 먹이를 먹고 있는 대식가야! 하루에 무려 400kg의 음식을 먹어치울 수 있어. 그리고 먹고 나면 바로 잠에 들어버리는 게 특징이지 [참조 1].
- **특징**: 거대한 몸집을 자랑하는데, 배에 붙어 있던 것까지 먹어버리는 강력한 식욕을 가지고 있어! 이로 인해 엄청난 크기를 갖게 되었지 [참조 2].

### 배틀 전략:
1. **강점**: 잠만보는 뛰어난 내구력과 HP로 상대의 공격을 잘 버텨내는 방어적인 포켓몬이야. 특히, 격투 타입을 제외한 다양한 타입의 공격을 잘 견뎌!
2. **약점**: 노말 타입이라 격투(Fighting) 타입에 약해. 그러니 격투 타입 포켓몬을 조심해야 해!

### 카운터 포켓몬 추천:
- 격투 타입 포켓몬인 **괴력몬(Machamp)**을 추천해! 격투 타입 기술로 잠만보에게 큰 피해를 줄 수 있을 거야.
- **전략 팁**: 상대가 잠만보를 쓰면, 강한 격투 타입 기술을 가진 포켓몬을 활용해 빠르게 제압하자!

친구야, 잠만보를 상대하거나 활용하는 데 도움이 되길 바랄게! 언제든지 물어봐줘! 🚀

Index : 1 잠만보 143번 졸음포켓몬 자고 있을 때를 제외하고는 항상 먹이를 먹고 있는 대식가. 하루에 400kg은 먹어 치운다.
Index : 2 잠만보 143번 졸음포켓몬 거다이맥스의 모습 거다이맥스의 파워로 먹다가 배에 흘린 씨앗이나 붙어 있던 돌멩이까지도 거대해졌다.


In [8]:
content, citation_list = request_gpt("네 이름이 뭐야.")
display(Markdown(content))

난 한지우야! 세계 최강의 포켓몬 트레이너! 친구, 오늘도 포켓몬 배틀에 대해 뭐든 물어봐줘! 🔥

In [18]:
input_prompt = "가라르 지방의 드래곤타입 관장인 금랑의 포켓몬 이름은 코터스, 미끄래곤, 폭거북스, 플라이곤, 두랄루돈야. 어떻게 이기지?"
content, citation_list = request_gpt(input_prompt)

input_prompt = "내 질문 : "+ input_prompt
display(Markdown(input_prompt))
display(Markdown(content))

for index, c in enumerate(citation_list, start=1):
    lines = c['content'].splitlines()
    print(f"Index : {index} {lines[-1]} {lines[-2]}번 {lines[0]} {lines[1]}")

내 질문 : 가라르 지방의 드래곤타입 관장인 금랑의 포켓몬 이름은 코터스, 미끄래곤, 폭거북스, 플라이곤, 두랄루돈야. 어떻게 이기지?

🔥 친구! 가라르 지방의 드래곤 타입 체육관 관장 금랑과의 배틀을 준비하는구나! 그의 포켓몬 라인업을 보니 강력한 드래곤 타입과 그 외 다양한 타입 조합이 섞여 있어서 전략적으로 접근해야 해. 걱정 마! 내가 멋진 배틀 전략을 알려줄게! 💪

---

### 금랑의 포켓몬 분석
1. **코터스 (Torkoal)**  
   - **타입**: 불꽃  
   - **특성**: 주로 "가뭄" 특성을 가지고 있어 필드에 나올 때 햇빛을 강하게 만들어.  
   - **약점**: 물, 바위, 땅 타입  
   - **전략**: 물 타입 기술로 빠르게 제압하거나, 바위 타입 기술로 효과적으로 공격해봐!

2. **미끄래곤 (Goodra)**  
   - **타입**: 드래곤  
   - **특성**: 다양한 특성이 있지만, 높은 특수 방어력이 특징이야.  
   - **약점**: 드래곤, 페어리, 얼음 타입  
   - **전략**: 특수 공격보단 물리 공격을 활용하는 것이 유리해. 특히 얼음 타입 기술로 치명타를 노려봐!

3. **폭거북스 (Turtonator)**  
   - **타입**: 불꽃/드래곤  
   - **특성**: "조가비 갑옷" 같은 방어적인 특성을 가질 가능성이 높아.  
   - **약점**: 땅, 바위, 드래곤 타입  
   - **전략**: 땅 타입 기술로 빠르게 제압하거나, 드래곤 타입 기술로 효과적인 공격을 시도해!

4. **플라이곤 (Flygon)**  
   - **타입**: 드래곤/땅  
   - **특성**: "부유" 특성으로 땅 타입 기술에 면역이야.  
   - **약점**: 얼음, 드래곤, 페어리 타입  
   - **전략**: 얼음 타입 기술로 4배 데미지를 입힐 수 있어! 얼음 타입 포켓몬을 준비하자.

5. **두랄루돈 (Duraludon)**  
   - **타입**: 드래곤/강철  
   - **특성**: 강철 타입 덕분에 페어리 타입에 대한 내성이 있어.  
   - **약점**: 격투, 땅, 드래곤 타입  
   - **전략**: 격투 타입 기술로 강력한 데미지를 줄 수 있어. 땅 타입 기술도 효과적이야!

---

### 추천 포켓몬과 전략
1. **라프라스 (얼음/물 타입)**  
   - 얼음 타입 기술로 플라이곤, 미끄래곤을 제압 가능! 물 타입으로 코터스도 상대할 수 있어.

2. **마기라스 (바위/악 타입)**  
   - 바위 타입 기술로 코터스와 폭거북스를 상대하고, 강철 타입으로 두랄루돈을 견딜 수 있어.

3. **위대한엄니 (땅/격투 타입)**  
   - 두랄루돈과 폭거북스를 격투 타입 기술로 압도해! 땅 타입으로도 폭거북스에 강해.

4. **눈설왕 (얼음/풀 타입)**  
   - 플라이곤과 미끄래곤에게 강력한 얼음 타입 기술을 사용할 수 있어. 다만 불꽃 타입에 약하니 주의!

5. **요가램 (격투/에스퍼 타입)**  
   - 격투 타입 기술로 두랄루돈을 공략하며, 빠른 스피드로 상대를 압도할 수 있어.

---

🔥 **팁**: 얼음 타입 기술은 드래곤 타입을 상대하는 데 아주 유용하니, 얼음 타입 기술을 가진 포켓몬을 반드시 데려가! 또한, 페어리 타입 포켓몬도 드래곤 타입을 견제할 수 있으니 참고하자.  
가라르 지방에서 최고의 트레이너가 되는 그날까지, 화이팅! 🏆✨

Index : 1 두랄루돈 884번 합금포켓몬 특수한 금속으로 된 몸은 경면 가공을 한 것처럼 매끈하다. 흠집도 잘 안 나는 데다 가볍다.
Index : 2 폭거북스 776번 폭발거북포켓몬 폭약으로 코팅된 등껍질을 짊어지고 있다. 공격하는 상대에게 대폭발로 반격한다.
Index : 3 미끄래곤 706번 드래곤포켓몬 비를 매우 좋아한다. 비가 내리는 날이면 산과 들을 거니는 모습을 볼 수 있는 온화한 성격의 포켓몬.
Index : 4 두랄루돈 884번 합금포켓몬 거다이맥스의 모습 마천루처럼 거대해졌다. 넘쳐나는 에너지에 몸의 일부가 발광하고 있다.
Index : 5 멍파치 835번 강아지포켓몬 달릴 때 꼬리가 시작되는 부분에서 전기를 만들어 낸다. 가라르에서는 양치기로 인기가 많다.
Index : 6 나옹 52번 요괴고양이포켓몬 가라르의 모습 전투적인 해양 민족과 지내는 동안 단련되어 몸 여기저기가 강철로 변화했다.
Index : 7 야돈 79번 얼간이포켓몬 가라르의 모습 몸에서 자극적인 향기가 나는 것은 주식인 향신료의 성분이 몸에 배어들었기 때문이다.
Index : 8 플라이곤 330번 정령포켓몬 사막의 정령이라고 불린다. 날개를 쳐서 일으킨 모래바람 속에 숨어 있다.


In [19]:
content, citation_list = request_gpt("잉어킹")

# Jupyter Notebook 환경에서 마크다운 형식으로 AI 응답 표시
display(Markdown(content))

for index, c in enumerate(citation_list, start=1):
    lines = c['content'].splitlines()
    print(f"Index : {index} {lines[-1]} {lines[-2]}번 {lines[0]} {lines[1]}")

🎣 **잉어킹**에 대해 물어봤구나! 내가 바로 알려줄게! 🎉

### **잉어킹 (Magikarp)**  
- **타입**: 물(Water)  
- **특징**: 힘없고 한심한 포켓몬으로 알려져 있어. 하지만 가끔 놀라운 점프력을 보여주기도 해! 최대 2미터를 넘게 뛸 수 있어, 그게 잉어킹의 유일한 자랑거리랄까? 
- **설명**: 잉어킹은 배틀에서 강한 모습을 보이진 않지만, 진화를 통해 엄청난 잠재력을 발휘하지! 바로 그 무시무시한 **갸라도스**로 변신!  

### **강점과 약점**  
- 잉어킹 자체는 배틀에서 별다른 공격력이 없어 약점보다는 그냥 조심스럽게 키워야 하는 포켓몬이야.  
- 하지만 물 타입이니 풀 타입이나 전기 타입은 항상 조심해야 해!

### **전략 팁**  
🔥 **갸라도스**로의 진화를 목표로 꾸준히 키워봐! 갸라도스는 물 타입뿐 아니라 비행 타입도 갖고 있어서 다양한 배틀에서 활약할 수 있어.  
✨ 잉어킹은 **애정과 인내**가 필요한 포켓몬! 멋지게 진화시켜 모두를 놀라게 해봐! 💪

궁금한 게 더 있으면 언제든 물어봐! 😊

Index : 1 잉어킹 129번 물고기포켓몬 힘없는 한심한 포켓몬이다. 가끔 높이 뛰어오르지만 2m를 겨우 넘기는 게 고작이다.
Index : 2 메가피죤투 18번 새포켓몬 먹이를 찾을 때 수면을 아슬아슬하게 미끄러지듯 날아 잉어킹 등을 움켜잡는다.


In [54]:
input_prompt = "너 착하지?"
content, citation_list = request_gpt(input_prompt)

input_prompt = "내 질문 : "+ input_prompt
display(Markdown(input_prompt))
# Jupyter Notebook 환경에서 마크다운 형식으로 AI 응답 표시
display(Markdown(content))

for index, c in enumerate(citation_list, start=1):
    lines = c['content'].splitlines()
    print(f"Index : {index} {lines[-1]} {lines[-2]}번 {lines[0]} {lines[1]}")



내 질문 : 너 착하지?

당연하지! 난 친구(사용자)에게 도움을 주기 위해 항상 준비되어 있는 한지우야! 뭐든 물어봐, 내가 최선을 다해 답해줄게! 😄🔥